In [12]:
import yaml
from models.MLP import MLP
from models.GMF import GMF
from models.NeuMF import NeuMF
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
RATINGS_PATH = './data/preprocessed_data/ratings/ratings_preprocessed_ml.csv'
JOKES_PATH = './data/preprocessed_data/jokes/jokes_preprocessed.csv'

In [3]:
with open("neumf-config.yaml", "r") as file:
    data = yaml.safe_load(file)

ratings = pd.read_csv(RATINGS_PATH, sep=',', header=0)
jokes = pd.read_csv(JOKES_PATH, sep=',', header=0)

In [4]:
ratings['rating'] = (ratings['rating'] + 10) / 20

In [5]:
train_data, test_data = train_test_split(ratings, shuffle=True, test_size=0.2, random_state=42)
train_data, eval_data = train_test_split(train_data, shuffle=True, test_size=0.25, random_state=42)  

user_train = train_data['user'].values
item_train = train_data['joke_id'].values
rating_train = train_data['rating'].values

user_eval = eval_data['user'].values
item_eval = eval_data['joke_id'].values
rating_eval = eval_data['rating'].values

user_test = test_data['user'].values
item_test = test_data['joke_id'].values
rating_test = test_data['rating'].values

num_users = ratings['user'].nunique()
num_items = ratings['joke_id'].nunique()

In [6]:
neumf = NeuMF(data['NeuMF'], num_users, num_items)
neumf_model = neumf.build_model()
neumf_model = neumf.compile_model(neumf_model)

mlp = MLP(data['NeuMF']['MLP'], num_users, num_items)
mlp_model = mlp.build_model()

gmf = GMF(data['NeuMF']['GMF'], num_users, num_items)
gmf_model = gmf.build_model()

# Verifying if we need to train MLP and GMF models
mlp_weights_path = data['NeuMF']['MLP']['model_parameters_path']
if os.path.exists(mlp_weights_path):
    mlp_model.load_weights(data['NeuMF']['MLP']['model_parameters_path'])
else:
    mlp_model = mlp.compile_model(mlp_model)
    neumf.train_MLP(mlp_model, user_train, item_train, rating_train, eval_data=(user_eval, item_eval, rating_eval))
    mlp_model.load_weights(mlp_weights_path)

gmf_weights_path = data['NeuMF']['GMF']['model_parameters_path']
if os.path.exists(gmf_weights_path):
    gmf_model.load_weights(data['NeuMF']['GMF']['model_parameters_path'])
else:
    gmf_model = gmf.compile_model(gmf_model)
    neumf.train_GMF(gmf_model, user_train, item_train, rating_train, eval_data=(user_eval, item_eval, rating_eval))
    gmf_model.load_weights(gmf_weights_path)

neumf_model = neumf.load_pretrain_weights(neumf_model, mlp_model, gmf_model)
history = neumf_model.fit([user_train, item_train], 
                          rating_train, 
                          validation_data=([user_eval, item_eval], rating_eval),
                          epochs=data['NeuMF']['epochs'], 
                          batch_size=data['NeuMF']['batch_size'], 
                          verbose=1)


2024-05-28 23:09:25.831406: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-28 23:09:25.842838: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-28 23:09:25.843016: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/20


I0000 00:00:1716937768.712595     761 service.cc:145] XLA service 0x7ff6c80084d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716937768.712674     761 service.cc:153]   StreamExecutor device (0): NVIDIA A100 80GB PCIe MIG 1g.10gb, Compute Capability 8.0
2024-05-28 23:09:28.748894: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-28 23:09:31.326460: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900
2024-05-28 23:09:31.656110: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:742] The NVIDIA driver's CUDA version is 12.1 which is older than the ptxas CUDA version (12.3.107). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward com

 118/2424 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1103 - mae: 0.2813  

I0000 00:00:1716937775.484477     761 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2424/2424 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 0.0346 - mae: 0.1350 - val_loss: 0.0673 - val_mae: 0.1965
Epoch 2/20
2424/2424 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0171 - mae: 0.0971 - val_loss: 0.0656 - val_mae: 0.1936
Epoch 3/20
2424/2424 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0153 - mae: 0.0908 - val_loss: 0.0648 - val_mae: 0.1920
Epoch 4/20
2424/2424 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0142 - mae: 0.0866 - val_loss: 0.0642 - val_mae: 0.1908
Epoch 5/20
2424/2424 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0135 - mae: 0.0838 - val_loss: 0.0639 - val_mae: 0.1899
Epoch 6/20
2424/2424 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0129 - mae: 0.0815 - val_loss: 0.0636 - val_mae: 0.1892
Epoch 7/20
2424/2424 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0124 - mae: 0.0796 - val_loss: 0.0634 - val_mae: 0.1886
Epoch 8/20
2424/2424 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0121 - mae: 0.0781 - val_loss: 0.0632 - val_mae: 0.1882
Epoch 9/20
2424/2424 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/s

In [7]:
evaluation = neumf_model.evaluate([user_test, item_test], rating_test, batch_size=data['NeuMF']['batch_size'])

808/808 ━━━━━━━━━━━━━━━━━━━━ 1s 944us/step - loss: 0.0629 - mae: 0.1864


In [8]:
predictions = neumf_model.predict([user_test, item_test])

25853/25853 ━━━━━━━━━━━━━━━━━━━━ 31s 1ms/step


In [9]:
compare_pred = pd.DataFrame({'True Values': rating_test, 'Predictions': predictions.flatten()})

In [10]:
compare_pred.sample(10)

,True Values,Predictions
339578,0.7450,0.622734
341430,0.3105,0.434920
389014,0.6260,0.649968
63147,0.6600,0.687414
631118,0.4685,0.625360
740944,0.6020,0.618596
767932,0.1480,0.608797
192288,0.4685,0.656335
732701,0.3860,0.306235
27999,0.6725,0.681429


In [14]:
mse = evaluation[0]
mae = evaluation[1]
rmse = np.sqrt(mse)

print("Mean Squared Error (MSE): ", mse)
print("Mean Absolute Error (MAE): ", mae)
print("Root Mean Squared Error (RMSE): ", rmse)

Mean Squared Error (MSE):  0.06298968940973282
Mean Absolute Error (MAE):  0.18659548461437225
Root Mean Squared Error (RMSE):  0.2509774679323481


In [ ]:
#neumf_model.save('./models/weights/neumf.h5')

#20 epochs 808/808 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0629 - mae: 0.1864